In [13]:
import os
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split

# 定义数据目录和文件路径
data_dir = "C:/Users/Rou/Desktop/Identifying-Pulse-through-Skin-Tone-Changes"
skin_pixel_data_path = os.path.join(data_dir, "HSV_output", "skin_pixel_changes.csv")

# 加载数据
data = pd.read_csv(skin_pixel_data_path)

# 转换日期和类别数据
data['量測日期時間'] = pd.to_datetime(data['量測日期時間']).map(datetime.toordinal)
data['性別'] = LabelEncoder().fit_transform(data['性別'])

# 特征选择
features = ['量測日期時間', '量測年齡', '性別', 'chinLip', 'forehead', 'leftCheek', 'lowerLip', 'nose', 'noseBridge', 'noseTip', 'rightCheek', 'upperLip']
target = ['心率']

# 离散化目标变量
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
data[target] = est.fit_transform(data[target])

# 标准化特征
scaler = StandardScaler()
X = scaler.fit_transform(data[features])
# Assuming 'target' is a list of column names, and you're processing a single target variable
y = data[target].values.ravel()  # Convert DataFrame column to NumPy array and then flatten it


# 数据分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
num_classes = data[target].nunique()
print("Number of classes:", num_classes)


Number of classes: 心率    10
dtype: int64


In [3]:
import torch

# Assuming X_train is a numpy array, directly convert it to a Tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)



C:\Users\Rou\AppData\Local\Temp\ipykernel_14500\725870678.py:4: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: 找不到指定的模組。 (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)


In [6]:
import torch
from momentfm import MOMENTPipeline

# 创建一个简单的随机Tensor作为输入
test_input = torch.randn(10, X_train_tensor.shape[1], dtype=torch.float32)  # 保持与X_train相同的特征数量

# 尝试初始化模型
try:
    model = MOMENTPipeline.from_pretrained(
        "AutonLab/MOMENT-1-large",
        model_kwargs={
            "task_name": "classification",
            "n_channels": test_input.shape[1],
            "num_class": 10
        }
    )
    model.init()
    print("Model initialized successfully with test input.")
except Exception as e:
    print("Error during model initialization:", e)


Model initialized successfully with test input.


In [12]:
import torch
from momentfm import MOMENTPipeline
# 正确的调用方式，指定了数据类型和设备
##tensor = torch.empty((10, 10), dtype=torch.float32, device='cpu')

# 如果X_train是numpy数组
##X_train_tensor = torch.empty(X_train, dtype=torch.float32, device='cpu')
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device='cpu')

# 确认Tensor的形状符合模型的输入要求
print("Shape of X_train_tensor:", X_train_tensor.shape)

n_channels = X_train_tensor.shape[1]  # 特征数量

try:
    model = MOMENTPipeline.from_pretrained(
        "AutonLab/MOMENT-1-large",
        model_kwargs={
            "task_name": "classification",
            "n_channels": n_channels,  # Start with minimal necessary parameters
            "num_class": num_classes # 确保这里的num_class正确
        }
    )
    model.init()
    print("Model initialized successfully.")
except Exception as e:
    print("Failed to initialize the model:", e)

    model.init()
    print("Model initialized successfully.")
except Exception as e:
    print("Failed to initialize the model:", e)


Shape of X_train_tensor: torch.Size([1635192, 12])
Failed to initialize the model: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)



TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
